In [2]:
#Cargar librerías
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [3]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [4]:
transformer=transforms.Compose([transforms.Resize((150,150)),transforms.RandomHorizontalFlip(),transforms.ToTensor(),transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])])#Preprocesamiento de los datos, se cambia el tamaño a un estandar determinado y se agrega un flip aleatorio de 50%
#para que en cada epoch haya variación en el dataset

In [5]:
#Cargar Dataloader
train_path = 'C:/Users/luisa/Desktop/11/Diagnostico asistido por Computadora/Proyecto final/BreaKHis 400X/train'
test_path = 'C:/Users/luisa/Desktop/11/Diagnostico asistido por Computadora/Proyecto final/BreaKHis 400X/test'

train_loader=DataLoader(torchvision.datasets.ImageFolder(train_path,transform=transformer),
batch_size=50, shuffle=True
)

test_loader=DataLoader(torchvision.datasets.ImageFolder(train_path,transform=transformer),
batch_size=15, shuffle=True
)

In [6]:
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)

['benign', 'malignant']


In [7]:
#Red Neuronal Convolutiva CNN
class ConvNet(nn.Module):
    def __init__(self,num_classes=2):
        super(ConvNet,self).__init__()

    #Tamanno del output despues de convolucion con filtros

        #Forma de la netrada #(batch_size,3,150,150)
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        self.bn1=nn.BatchNorm2d(num_features=12)
        self.relu1=nn.ReLU()

        self.pool=nn.MaxPool2d(kernel_size=2)

        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        self.relu2=nn.ReLU()
    #shape=(batch_size,20,75,75)

        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.bn3=nn.BatchNorm2d(num_features=32)
        self.relu3=nn.ReLU()
    #shape=(batch_size,32,75,75)


        self.fc=nn.Linear(in_features=32*75*75, out_features=num_classes)

    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)

        output=self.pool(output)

        output=self.conv2(output)
        output=self.relu2(output)

        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)

        output=output.view(-1,32*75*75)
        output=self.fc(output)
        return output



In [8]:
model=ConvNet(num_classes=2).to(device)

#Optimizador de funcion de perdida
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

num_epochs=10

#Calculando el tamano de entrenamiento y testeo
train_count=len(glob.glob(train_path+'/**/*.png'))
test_count=len(glob.glob(test_path+'/**/*.png'))
print(train_count,test_count)

1148 545


In [10]:
#Entrenamiento y guardado de mejor modelo
best_accuracy=0.0

for epoch in range(num_epochs):
    #Evaluacion y entrenamiento
    model.train()
    train_accuracy=0.0
    train_loss=0.0

    for i, (images,labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)

        train_accuracy+=int(torch.sum(prediction==labels.data))
        train_accuracy=train_accuracy/train_count
        train_loss=train_loss/train_count
        


    #Testeo
    model.eval()
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):

        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))

        test_accuracy=test_accuracy/test_count

    print('Epoch: '+str(epoch)+' Train Loss: '+str(int(train_loss))+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))

#Save best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy


KeyboardInterrupt: 